## Housing: XGB Meta-regressor with LGBM, SVR, RF --> score: 0.12090

This notebook is mainly oriented towards ML workflow using sklearn pipelines.  In order to modify certain transformers for pipeline use, techniques from http://flennerhag.com/2017-01-08-Recursive-Override/ are used and can be viewed in the attached supporting_files. Models were modified slightly to include baysian optimizers with mixed results. You can find additional notebooks and documents at my github: https://github.com/jander081.

This dataset contains an abundance of missing values. It is not enough, however, to simple choose an imputation technique and proceed. Missing values need to be explored in order to determine the best handling method. For example, many of the null values are related (i.e. no basement = no finished, basement base, etc). Additionally, it cannot be assumed that a feature with a high percentage of null values should be removed. The pool feature is an example of this as it's null values exceed 99.6%. However, since null values indicate the absence of a pool, thee presence of a pool can carry additional price information that should be included in the model.  

__Approach:__

1. Missing values 
2. Engineer features
3. Pipelines
4. Stack models

In [ ]:
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/supporting-files/housing_code.py", dst = "../working/housing_code.py")
copyfile(src = "../input/supporting-files/housing_imports.py", dst = "../working/housing_imports.py")
copyfile(src = "../input/supporting-files/housing_models.py", dst = "../working/housing_models.py")
copyfile(src = "../input/github-code/pandas_feature_union.py", dst = "../working/pandas_feature_union.py")
copyfile(src = "../input/github-code/__init__.py", dst = "../working/__init__.py")

# import all our functions
from housing_imports import *
from housing_code import *
from housing_models import *
from pandas_feature_union import *
from __init__ import *

In [ ]:
# PRESERVE THE HOME IDS IN FROM THE TEST SET. MERGE THE TWO SETS FOR PROCESSING
csv_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv').drop_duplicates()
csv_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv').drop_duplicates()
y = csv_train.iloc[:, -1]
data = pd.concat([csv_train.iloc[:, 1:-1], csv_test.iloc[:, 1:]], axis=0)
data.reset_index(drop=True, inplace=True)
# print(data.shape)
house_id = pd.DataFrame(csv_test.Id)

### EDA -> MISSING VALUES AND TARGET FEATURE

In [ ]:
# COUPLE DIFFERENT TYPES OF NULL VALUES
null_columns=data.columns[data.isnull().any()]
data[null_columns].isnull().sum().head() # ABBREVIATED

In [ ]:
# NULL VALUES FOR GARAGE AND BASEMENT FOLLOW A TREND
import missingno as msno
cats = TypeSelector(np.object).fit_transform(data)
nulls = cats[cats.columns[cats.isnull().any()]]
msno.matrix(nulls);

In [ ]:
# CREATE ABSOLUTE TIME FEATURES -> EASY ANSWER. TACKING ON A STRING ENSURES
# THAT THE NUMBER IS NOT ACCIDENTLY CONVERTED BACK TO A NUMERICAL LATER
# ALSO KEEP NUMERICALS FOR BINNING
# GarageYrBlt REMOVED

years = ['YearBuilt', 'YearRemodAdd', 'YrSold']

for colname in years:
        data[colname + '_cat'] = data[colname].apply(lambda x: x if np.isnan(x) else 'year_' + str(int(x)))

# PERMANENTLY TRANSFORM TO CAT - NOT NUMERICAL IN NATURE
data['MSSubClass'] = data.MSSubClass.apply(lambda x: 'class ' + str(x)) 


In [ ]:
from tqdm import tqdm
tqdm.pandas()
# NOT MUCH DIFFERENCE BETWEEN GarageYrBlt AND YearBuilt. BASICALLY, 
# THE VALUES DIFFER IF A GARAGE WAS ADDED. A NEW INDICATER FEATURE IS 
# MADE AND THE COLUMN IS DROPPEED

for i in tqdm(range(0, data.shape[0])):
    if np.isnan(data.GarageYrBlt[i]):
        year = data.YearBuilt[i]
        data.GarageYrBlt[i] = year
        
data.GarageYrBlt = data.GarageYrBlt.apply(lambda x: int(x))

new_feat = []
for i in range(0, data.shape[0]):
    if data.GarageYrBlt[i] == data.YearBuilt[i]:
        new_feat.append(0)
    else:
        new_feat.append(1)

# CREATE AN INDICATOR DATAFRAME. THIS HELPS AVOID CONFUSION DURING FINAL
# PREPROCESSING

    
data['Garage_added'] = new_feat
data['Garage_added'] = data['Garage_added'].astype("bool")
data.drop(['GarageYrBlt'], axis=1, inplace=True)

In [ ]:
# CREATE AN INDICTATOR FOR REMODEL
new_feat = []
for i in range(0, data.shape[0]):
    if data.YearBuilt[i] == data.YearRemodAdd[i]:
        new_feat.append(0)
    else:
        new_feat.append(1)
        
data['Remodeled'] = new_feat
data['Remodeled'] = data['Remodeled'].astype("bool")
# CONVERT A FEW MORE BOOLS FOR FUN
data['paved_street'] = data.Street.apply(lambda x: 1 if x == 'Pave' else 0).astype('bool')
data['central_air'] = data.CentralAir.apply(lambda x: 1 if x == 'Y' else 0).astype('bool')
data.drop(['Street', 'CentralAir'], axis=1, inplace=True)
# WE'LL LEAVE YearRemodAdd FOR FEATURE SELECTION 

In [ ]:
# ADDING RELATIVE REFACTORED TIME FEATURES
import datetime
current = datetime.date.today()
# print(current.year)
data['sold_delta'] = current.year - data['YrSold'] 
data['built_delta'] = current.year - data['YearBuilt'] 
data['remodel_delta'] = current.year - data['YearRemodAdd'] 

In [ ]:
# GIVEN THE NUMBER OF NULLS, DISTRIBUTION OF VALUES, AND RELATIONSHIP
# BETWEEN FEATURES, SOME NULLS WILL BE FILLED WITH MODE AND SOME WILL
# BE CONVERTED TO A NEW CATEGORY, "NONE"

none_list = ['Alley', 
       'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 
       'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageQual', 'GarageCond', 'Fence', 'MiscFeature', 'PoolQC']

for colname in none_list:
    data[colname].fillna('None', inplace=True)
    

In [ ]:
# LOOKS READY FOR THE PIPELINES
null_columns=data.columns[data.isnull().any()]
print(data[null_columns].isnull().sum())


### Preprocessing Pipeline

The remain preprocessing has been abstracted using sklearn pipelines and a mix of transformers. Some of the transformers are lesser known (i.e. SoftImpute), some are wrapped standard transformers (i.e. KBins, StandardScaler), and some are custom (i.e. RegImpute, FreqFeatures). All of the wrapped classes can be viewed in attachments. The main purpose for the wrappers was to allow for pipeline use. 

In [ ]:
# I'VE ABSTRACTED AWAY MOST OF THE PREPROCESSING AND BASIC ENGINEERING. THE
# TRANSFORMERS ARE VIEWABLE IN ATTACHMENTS. SOME OF THE TRANSFORMERS ARE SIMPLY
# WRAPPERS THAT ALLOW THE TRANSFORMERS TO FUNCTION IN AN SKLEARN PIPELINE.

transformer_list=[
        ("binned_features", make_pipeline(
                        TypeSelector(np.number),
                        StandardScalerDf(),
                        SoftImputeDf(),
                        SelectFeatures(),
                        KBins()
        )),
        ("numeric_features", make_pipeline(
                            TypeSelector(np.number),
                            StandardScalerDf(),
                            SoftImputeDf()
        )),
        ("categorical_features", make_pipeline(
                             TypeSelector(np.object),
                             RegImpute() 
        )),
        ("frequency_features", make_pipeline(
                         TypeSelector(np.object),
                         RegImpute(),
                         SelectFeatures(val_count=15, categorical=True),
                         FreqFeatures()
        )),
        ("boolean_features", make_pipeline(
                         TypeSelector(np.bool_),
                         RegImpute(regex=False) 
        ))  ]

__PandasFeatureUnion__ simply corrects for the numpy array output of sklearn FeatureUnion. It can be downloaded from Github at: https://github.com/marrrcin/pandas-feature-union

__QuickPipeline__ is an awesome preprocessing pipeline that can be found at: https://github.com/Mottl/quickpipeline.

QuickPipeline is slightly modified below to work in an sklearn pipeline. Basically, it inherits TransformerMixin and is given fit & transform functions rather than only fit_transform.

In [ ]:
preprocess_pipeline = make_pipeline(
    PandasFeatureUnion(transformer_list),
    QuickPipeline_mod()  )
    

In [ ]:
X = preprocess_pipeline.fit_transform(data)

In [ ]:
# I TRY TO KEEP THINGS IN PANDAS MOST OF THE TIME. THIS COMES IN HANDY WHEN 
# ANALYZING FEATURES LATER ON. 
X.head(2)

In [ ]:
# TAKING THE LOG OF THE TARGET CORRECTS FOR A RIGHT SKEW. HOWEVER, THE LOGNORMAL
# CREATES A SLIGHT LEFT SKEW
# NORMALIZATION IS FOR COMPARISON PURPOSES ONLY
# SEPARATED FOR COMPARISON ONLY

y_norm = y.apply(lambda x: (x - y.mean()) / y.std())
y_box, lambda_ = boxcox(y) # need the lambda to eventually reverse the transformation
y_box_norm = pd.DataFrame(y_box).apply(lambda x: (x - y_box.mean()) / y_box.std())
sns.kdeplot(y_norm, label='normal')
sns.kdeplot(np.log(y_norm) + 4, label='lognormal')
sns.kdeplot(np.ravel(y_box_norm) - 4, label='box-cox')
plt.legend();

# THE LEAST SKEWED IS THE BOX COX TRANSFORMATION
y, lambda_ = boxcox(y)


In [ ]:
# MAKE SURE THE BOX COX IS REVERSIBLE. THE POWER TRANSFORM AVAILABLE 
# THROUGH SKLEARN DOES NOT SEEM TO PROVIDE A LAMBDA AND IS NOT EASILY REVERSIBLE

def invboxcox(y,ld):
    if ld == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(ld*y+1)/ld))

#  LITTLE TEST TO MAKE SURE
# test = csv_train.iloc[:, -1][:100]; print(test[:3])
# y_box_test, lambda_test = boxcox(test); print(pd.Series(y_box_test)[:3])
# # Add 1 to be able to transform 0 values
# test_rev = invboxcox(y_box_test, lambda_test);print(pd.Series(test_rev).apply(lambda x: np.int64(x))[:3])


In [ ]:
X_test = X.iloc[1460:, :]
X_ = X.iloc[:1460, :]
print(X_.shape);print(X_test.shape);print(y.shape)
X_train = X_
y_train = y

### Stacked Models

You'll notice the __Bayes__ wrapper around each model. If you look at models.py, you'll see that a bayes optimizer is embedded in each model. Intervals are defined using dictionaries. 

In [ ]:
xgb_params

While bayesian hyperparameter optimization is an exciting field, it is a bit harder to implement than one may initially assume. Simply defining large intervals for the optimizer to use will generally result in poorer performance. I explored this in detail in my master's thesis. Basically, one can spend a lifetime trying to understand a given hyperparameter's affect on model behavior, only to see the model completely misbehave or respond differently given another hyperparameter's interval. The best method I've found (so far) is to obtain a decent understanding of the given hyperparameter and default setting. Using a smaller random subset of data, begin testing different intervals using cross validation scores. I'll also search online for hyperparameters used for similarly sized data and test intervals based off values used by other data scientists. 

I'm also exploring combining a grid search method (for intervals) with bayesian techniques.

In [ ]:
# REGRESSORS
lgb = BayesLGBMRegressor()
# LGBM WOULD NOT ACCEPT A DICT AS A HYPERPARAMETER - I'D NEED TO EXPLORE THIS MORE, USE **kwargs
svr = BayesSVR(intervals=svr_params)
rf = BayesRandomForest(intervals=rf_params)
regressors = [lgb, svr, rf]

# META-REGRESSOR
meta = BayesXGBRegressor(intervals=xgb_params)

In [ ]:
# RELATIVELY STRAIGHTFORWARD. EASY TO DISSECT AND UNDERSTAND THE RELATIONSHIP
# BETWEEN THE META-MODEL AND BASE MODELS.

ensemble = StackingCVRegressorAveraged(regressors=regressors, 
                                       meta_regressor=meta)

In [ ]:
ensemble.fit(X_train, y_train)

In [ ]:
y_pred = ensemble.predict(X_test)

In [ ]:
# REVERSE BOX COX TRANSFORMATION
labels = pd.DataFrame(invboxcox(y_pred, lambda_)).apply(lambda x: np.float64(round(x, 2)))

In [ ]:
submit = pd.concat([house_id, labels], axis=1)

submit.set_index('Id', inplace=True)

submit.rename(columns={0: 'SalePrice'}, inplace=True)
submit.head()

In [ ]:
# submit.to_csv('preds/predictions5.csv', index=True)

## Extra

 __Notes__: While looking over the data description, it seemed like a lot of the categorical features could be converted to ordinal for additional 
features. I've included the transformations below. The ensemble performance unfortunately did not improve. However, that could easily be the result of the optimier intervals or ensemble method (or a host of other factors). It seems that adding an ordinal aspect to a categoical feature (when appropriate) should capture more information for model use. I'd love to explore feature engineering with this dataset further, but unfortunately I need to move on. Perhaps someone can build upon the code provided below. Anyways, __happy kaggling!__

In [ ]:
# SOME OF THE CATEGORICALS CAN ALSO BE REPRESENTED AS ORDINALS
# data['BsmtQual'].value_counts(dropna=False)
# cat_to_ordinal_1 = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 
#                     'Ex': 5}

# cat_to_ordinal_2 = {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 
#                     'ALQ': 5, 'GLQ': 6}

# Functionality = {'Sal Salvage': 0, 'Sev Severely': 1, 'Maj2': 2, 'Maj1': 3,
#                 'Mod': 4, 'Min2': 5, 'Min1': 6, 'Typ': 7}

# exposure = {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}


In [ ]:
# cat_to_num_1 = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC',
#                'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond',
#                'PoolQC']

# cat_to_num_2 = ['BsmtFinType1', 'BsmtFinType2']


In [ ]:
# for feat in cat_to_num_1:
#     data[feat].fillna(data[feat].mode()[0], inplace=True)
#     data[feat + '_num'] = data[feat].map(cat_to_ordinal_1)

# for feat in cat_to_num_2:
#     data[feat].fillna(data[feat].mode()[0], inplace=True)
#     data[feat + '_num'] = data[feat].map(cat_to_ordinal_2)

# data['bsmt_exp_num'] =  data['BsmtExposure'].map(exposure)


__References:__

https://github.com/travisbrady/py-soft-impute

http://arxiv.org/abs/1410.2596 
Awesome paper. I break soft thresholding down fairly well in my thesis available at my github (use TOC - it's long). I'll also try to include a notebook that really breaks down solft thresholding using numpy.linalg. 

__other kagglers:__ 

https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

https://www.kaggle.com/neviadomski/how-to-get-to-top-25-with-simple-model-sklearn